### This is the ipython transcript for the `RAPHE_reward.py`

In [1]:
%matplotlib inline

In [2]:
import os;
os.chdir('/home/gergely/code/analysis/analysis-scripts/')
import sys;
import copy;

import cPickle as pickle;

import pandas as pd;

import numpy as np;
import scipy.stats as stats;

import matplotlib;
matplotlib.use("pdf");

import matplotlib.pyplot as pp;
import matplotlib.backends.backend_pdf as pdf;
import matplotlib.gridspec as gs;

import lab;
import lab.analysis.imaging_analysis as ia;
import lab.analysis.behavior_analysis as ba;
import lab.analysis.reward_analysis as ra;

import lab.classes.exceptions as exc;

import lab.plotting.plotting_helpers as plotTools;

import VIP_common as vc;
matplotlib.rcParams['pdf.fonttype'] = 42;
matplotlib.rcParams['ps.fonttype'] = 42;

/home/gergely/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/gergely/anaconda2/lib/python2.7/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)
/home/gergely/code/analysis/lab/misc/lfp_helpers.py:16: UserWarning: rhd package not found, lab.misc.lfp_helpers.py will not work
  warnings.warn("rhd package not found, lab.misc.lfp_helpers.py will not work")


In [3]:
def calcMaxConsecutive(boolArr):
    conSize=np.diff(
        np.where(
            np.concatenate(
                ([boolArr[0]], boolArr[:-1]!=boolArr[1:], [True])))[0]);
    if(conSize.size<=0):
        return 0;
    
    return np.max(conSize[::2]);

In [4]:
def bootstrapTraceDiff(df, groupbyCols, g1Key, g2Key, dataCol, timeCol, minSamples=5,
                       iterations=3000, ciLevel=0.95):
    groups=df.groupby(groupbyCols);
    if(len(groups)!=2):
        print("wrong number of groups");
        return pd.DataFrame();
    try:
        g1=groups.get_group(g1Key);
        g2=groups.get_group(g2Key);
    except KeyError:
        print("bad keys");
        return pd.DataFrame();
    
    g1NS=len(g1);
    g2NS=len(g2);
    
    numSamples=np.min([g1NS, g2NS]);
    if(numSamples<minSamples):
        return pd.DataFrame();
    
    ciPLow=(1-ciLevel)/2.0*100;
    ciPHigh=ciLevel*100.0+ciPLow;
    
    time=g1[timeCol].iloc[0];
    
    g1Data=np.vstack(g1[dataCol]);
    g2Data=np.vstack(g2[dataCol]);
    
    g1RSel=np.random.randint(g1NS, size=(iterations, numSamples));
    g2RSel=np.random.randint(g2NS, size=(iterations, numSamples));
    
    g1RS=np.nanmean(g1Data[g1RSel], 1);
    g2RS=np.nanmean(g2Data[g2RSel], 1);
    
    gDiffRS=g2RS-g1RS;
    
    gDiffRSPre=np.nanmean(gDiffRS[:, time<0], 1);
    gDiffRSPost=np.nanmean(gDiffRS[:, time>=0], 1);
    
    gDiffRSPPDiff=gDiffRSPost-gDiffRSPre;
    
    result=dict();
    
    result["diffTraceMed"]=[np.nanmedian(gDiffRS, 0)];
    result[timeCol]=[time];
    result["ppDiffTraceMed"]=np.nanmedian(gDiffRSPPDiff);
    result["ppDiffTraceCIH"]=np.nanpercentile(gDiffRSPPDiff, ciPHigh, interpolation="higher");
    result["ppDiffTraceCIL"]=np.nanpercentile(gDiffRSPPDiff, ciPLow, interpolation="lower");
    result["ppDiffTraceSig"]=((result["ppDiffTraceCIH"]<0)*-1)+((result["ppDiffTraceCIL"]>0)*1);
    
    return pd.DataFrame(result);

In [5]:
expSet=lab.ExperimentSet("raphe.sql");
mouseIDs=["gtr5_01"];

# hrExpGroups=vc.loadHRExpGrp(expSet, mouseIDs, minLicks=10);
rfExpGroups=vc.loadExptsByType(expSet, mouseIDs, "randomForaging", channel="Ch1");
# rtExpGroups=vc.loadExptsByType(expSet, mouseIDs, "runTraining", channel="Ch1")
# orfExpGroups=vc.loadExptsByType(expSet, mouseIDs, "operantRandomForaging");

# salExpGroup=vc.loadExptsByType(expSet, mouseIDs, "salience")["imaging"];


imExpList=list(rfExpGroups["imaging"])
# imExpList.extend(list(rtExpGroups["imaging"]))
# imExpList.extend(list(hrExpGroups["imaging"]));
# imExpList.extend(list(orfExpGroups["imaging"]));
imExpGroup=lab.classes.ExperimentGroup(imExpList);

expProps=vc.loadExpPropsManualLabel(imExpGroup);
expProps["mouseID"]=map(lambda expt: expt.parent.get("mouseID"), expProps["expt"]);
expProps["expType"]=map(lambda expt: expt.get("experimentType"), expProps["expt"]);
expProps["frameP"]=map(lambda expt: expt.frame_period(), expProps["expt"]);



In [6]:
expProps["keep"]=True;
expProps.set_index(["expType", "condition"], inplace=True);
# expProps.loc[("randomForaging", "A"), "keep"]=False;
expProps=expProps[np.array(expProps["keep"])].reset_index();

imExpGroup=lab.ExperimentGroup(list(expProps["expt"]));

In [7]:
expProps

,expType,condition,condition_day,condition_day_session,expt,mouseID,frameP,keep
0,randomForaging,A,A_0,A_0_0,[[]],gtr5_01,0.16673,True
1,randomForaging,A,A_1,A_1_0,[[]],gtr5_01,0.16673,True
2,randomForaging,A,A_2,A_2_0,[[]],gtr5_01,0.16673,True
3,randomForaging,A,A_2,A_2_1,[[]],gtr5_01,0.16673,True
4,randomForaging,A,A_3,A_3_0,[[]],gtr5_01,0.16673,True
5,randomForaging,A,A_3,A_3_1,[[]],gtr5_01,0.16673,True
6,randomForaging,A,A_3,A_3_2,[[]],gtr5_01,0.16673,True


In [8]:
stimuli=["running_stop", "water"];
data=vc.getData(imExpGroup, stimuli, channel="Ch1", imPreTime=3, imPostTime=3, 
                behaviorDataKeys=["water", "velocity", "licking"]);                

data=data.merge(expProps, on="expt");
data.drop(["ROI", "expt"], 1, inplace=True);
data["conseqTThresh"]=np.ceil(np.array(0.3/data["frameP"]));

data.rename(columns={"condensedROI": "cROI"}, inplace=True);

data["preRest"]=map(lambda v, t, tth: calcMaxConsecutive(v[t<-0.75]>1)<=tth, 
                    data["velocity"], data["velocityTime"], data["conseqTThresh"]);
data["postRest"]=map(lambda v, t, tth: calcMaxConsecutive(v[t>=0.75]>1)<=tth, 
                     data["velocity"], data["velocityTime"], data["conseqTThresh"]);
data["preRun"]=map(lambda v, t, tth: calcMaxConsecutive(v[t<-0.75]<1)<=tth, 
                   data["velocity"], data["velocityTime"], data["conseqTThresh"]);
data["postRun"]=map(lambda v, t, tth: calcMaxConsecutive(v[t>=0.75]<1)<=tth, 
                    data["velocity"], data["velocityTime"], data["conseqTThresh"]);
data["rewarded"]=map(lambda w: np.nansum(w)>0, data["water"]);
data["rewardedPre"]=map(lambda w, t: np.nansum(w[t<0])>0, data["water"], data["waterTime"]);
data["rewardedPost"]=map(lambda w, t: np.nansum(w[t>=0])>0, data["water"], data["waterTime"]);

stims=np.array(data["stimulus"]);
preRest=np.array(data["preRest"]);
postRest=np.array(data["postRest"]);
preRun=np.array(data["preRun"]);
postRun=np.array(data["postRun"]);

In [9]:
data["valid"]=(((stims=="running_start") & preRest & postRun) |
               ((stims=="running_stop") & preRun & postRest));

In [10]:
data["licked"]=map(lambda l, t: np.nansum(l[t>0])>0, data["licking"], data["lickingTime"]);
data["actDiff"]=map(lambda a, t: np.nanmean(a[t>=0])-np.nanmean(a[t<0]), 
                    data["activity"], data["time"]);

dataGroups=data.groupby(["stimulus", "rewarded", "licked"]);
rewardedData=dataGroups.get_group(("water", True, True)).append(
    dataGroups.get_group(("running_stop", True, True)));
rewardedData.loc[rewardedData["stimulus"]=="running_stop", "stimulus"]="running_stop_rewarded";
with open("/data/gergely/workingData/raphe/reward/rewarded_traces_new.pkl", "wb") as fh:
    pickle.dump(rewardedData, fh);

In [11]:
data.set_index(["stimulus", "rewarded", "valid"], inplace=True);
runStopData=data.loc[("running_stop", True, True)].reset_index();
runStopData=runStopData.append(data.loc[("running_stop", True, False)].reset_index());
runStopData=runStopData.append(data.loc["running_stop", False, True].reset_index());

/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  
/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  after removing the cwd from sys.path.


In [12]:
runStopRDiffROIs=runStopData.groupby(["expType", "condition_day", "cROI"])

In [13]:
len(runStopData["expType"])

16721

In [15]:
group = runStopRDiffROIs
len(group)

1898

In [12]:
runStopData.columns

Index([u'stimulus', u'rewarded', u'valid', u'activity', u'stimStart', u'time',
       u'cROI', u'water', u'lapNum', u'waterTime', u'velocity',
       u'velocityTime', u'licking', u'lickingTime', u'expType', u'condition',
       u'condition_day', u'condition_day_session', u'mouseID', u'frameP',
       u'keep', u'conseqTThresh', u'preRest', u'postRest', u'preRun',
       u'postRun', u'rewardedPre', u'rewardedPost', u'licked', u'actDiff'],
      dtype='object')

In [13]:
runStopRDiffROIs=runStopData.groupby(["expType", "condition_day", "cROI"]).apply(
    bootstrapTraceDiff, ["rewarded"], False, True, "activity", "time", 
    minSamples=5, iterations=3000, ciLevel=0.99);

In [19]:
temp=runStopData[["stimulus", "rewarded", "stimStart", "mouseID", "expType", "valid", "condition_day_session"]].drop_duplicates()

In [22]:
temp.groupby(["expType", "mouseID", "condition_day_session", "stimulus", "valid", "rewarded"]).size()

expType         mouseID  condition_day_session  stimulus      valid  rewarded
randomForaging  gtr5_01  A_0_0                  running_stop  False  True        1
                                                              True   False       4
                                                                     True        1
                         A_1_0                  running_stop  False  True        2
                                                              True   False       5
                                                                     True        1
                         A_2_0                  running_stop  True   False       7
                                                                     True        1
                         A_2_1                  running_stop  False  True        5
                                                              True   False       1
                         A_3_0                  running_stop  False  True        1
         

In [23]:
temp.groupby(["expType", "mouseID", "condition_day_session", "stimulus", "rewarded"]).size()

expType         mouseID  condition_day_session  stimulus      rewarded
randomForaging  gtr5_01  A_0_0                  running_stop  False       4
                                                              True        2
                         A_1_0                  running_stop  False       5
                                                              True        3
                         A_2_0                  running_stop  False       7
                                                              True        1
                         A_2_1                  running_stop  False       1
                                                              True        5
                         A_3_0                  running_stop  False       7
                                                              True        2
                         A_3_1                  running_stop  False       4
                                                              True        8
                 

In [24]:
runStopSigROIs.head()

,expType,condition_day,cROI,rewarded,level_4,diffCIH,diffCIL,diffMed,diffSig,numTraces,postCIH,postCIL,postMed,preCIH,preCIL,preMed,act,time
0,randomForaging,A_1,"(gtr5_01, , 1533238970022)",False,0,0.037009,-0.047108,-0.006255,0.0,5.0,0.142726,-0.044452,0.038584,0.162394,-0.071127,0.044428,"[0.0686456637033, 0.0429531522244, 0.006013281...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."
1,randomForaging,A_1,"(gtr5_01, , 1533239014386)",False,0,-0.065179,-0.292066,-0.152782,-1.0,5.0,-0.044083,-0.199846,-0.117630,0.091684,-0.026606,0.035612,"[-0.0370432945658, -0.00518844859535, 0.045031...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."
2,randomForaging,A_1,"(gtr5_01, , 1533239157834)",False,0,0.046194,-0.115919,-0.037133,0.0,5.0,0.036924,-0.102518,-0.033551,0.047001,-0.052405,0.003581,"[0.00482485352016, -0.00225862192442, 0.015333...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."
3,randomForaging,A_1,"(gtr5_01, , 1533239257298)",False,0,0.036696,-0.136239,-0.069292,0.0,5.0,0.037930,-0.079483,-0.009452,0.114596,-0.028619,0.056903,"[-0.029991666698, -0.00228174662395, 0.0184941...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."
4,randomForaging,A_1,"(gtr5_01, , 1533239272640)",False,0,0.112448,-0.126598,0.002144,0.0,5.0,0.019105,-0.043625,-0.017559,0.105231,-0.130272,-0.019703,"[0.0541157622252, 0.00652556922304, 0.01178112...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."


In [88]:
temp=runStopData[["stimulus", "rewarded", "stimStart", "mouseID", "expType", "condition_day_session"]].drop_duplicates()

In [90]:
temp.groupby(["expType", "mouseID", "condition_day_session", "stimulus", "rewarded"]).size()

expType         mouseID  condition_day_session  stimulus      rewarded
randomForaging  gtr5_01  A_0_0                  running_stop  False       58
                                                              True        97
dtype: int64

In [91]:
temp=runStopData[["stimulus", "rewarded", "stimStart", "mouseID", "expType", "valid", "condition_day_session"]].drop_duplicates()

In [92]:
temp.groupby(["expType", "mouseID", "condition_day_session", "stimulus", "valid", "rewarded"]).size()

expType         mouseID  condition_day_session  stimulus      valid  rewarded
randomForaging  gtr5_01  A_0_0                  running_stop  False  True        70
                                                              True   False       58
                                                                     True        28
dtype: int64

there are enough events

In [28]:
runStopRDiffROIs.reset_index(inplace=True);
runStopRDiffROIs.rename(columns={"diffTraceMed": "actResidual", 
                                 "ppDiffTraceCIH": "residualDiffCIH",
                                 "ppDiffTraceCIL": "residualDiffCIL",
                                 "ppDiffTraceMed": "residualDiffMed",
                                 "ppDiffTraceSig": "residualDiffSig"},
                        inplace=True);

In [29]:
runStopSigROIs=runStopData.groupby(["expType", "condition_day", "cROI", "rewarded"]).apply(
    vc.calcResponseMetrics, col="activity", xCol="time", iterations=3000, ciLevel=0.99, 
    minDataSize=5, returnDF=True);
runStopSigROIs.reset_index(inplace=True);
runStopSigROIs.rename(columns={"diffSigResShuffle": "diffSig", 
                               "traceMed": "act", 
                               "traceXVal": "time"},
                      inplace=True);

In [31]:
# not enough events. this is the remedy. the cells below should run
runStopSigROIs.set_index(["rewarded"], inplace=True)

In [32]:
plotROIs=runStopSigROIs.loc[False][["expType", "condition_day", "cROI", "diffSig", "act", "time"]].merge(
    runStopSigROIs.loc[True][["expType", "condition_day", "cROI", "diffSig", "act", "time"]], 
    on=["expType", "condition_day", "cROI"], suffixes=("URD", "RD"));

plotROIs=plotROIs.merge(runStopRDiffROIs[["expType", "condition_day", "cROI", 
                                          "actResidual", "time", "residualDiffSig"]],
                        on=["expType", "condition_day", "cROI"]);
                        
residualPlotTraces=plotROIs.groupby(["expType", "condition_day", 
                                     "residualDiffSig", "diffSigURD", "diffSigRD"]).apply(
                                         vc.aggregratePSTH, "actResidual", "time", 
                                         measureSuffix="", varMethod=stats.sem, varSuffix="SEM");
residualPlotTraces.rename(columns={"actResidual": "act",
                                   "actResidualSEM": "actSEM"}, inplace=True);
unRDPlotTraces=plotROIs.groupby(["expType", "condition_day", 
                                 "residualDiffSig", "diffSigURD", "diffSigRD"]).apply(
                                     vc.aggregratePSTH, "actURD", "timeURD", 
                                     measureSuffix="", varMethod=stats.sem, varSuffix="SEM");
unRDPlotTraces.rename(columns={"actURD": "act", "actURDSEM": "actSEM", "timeURD": "time"},
                      inplace=True);
rwdPlotTraces=plotROIs.groupby(["expType", "condition_day", 
                                 "residualDiffSig", "diffSigURD", "diffSigRD"]).apply(
                                     vc.aggregratePSTH, "actRD", "timeRD", 
                                     measureSuffix="", varMethod=stats.sem, varSuffix="SEM");
rwdPlotTraces.rename(columns={"actRD": "act", "actRDSEM": "actSEM", "timeRD": "time"},
                              inplace=True); 

/home/gergely/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [33]:
plotTraces=dict();
plotTraces["residual"]=residualPlotTraces;
plotTraces["unRD"]=unRDPlotTraces;
plotTraces["rwd"]=rwdPlotTraces;
plotColors=dict();
plotColors["residual"]="red";
plotColors["unRD"]="black";
plotColors["rwd"]="blue";
traceKeys=["residual", "unRD", "rwd"];

In [34]:
expTypeList=residualPlotTraces.index.get_level_values(0).unique();
cdList=residualPlotTraces.index.get_level_values(1).unique();
sigList=[-1, 0, 1];
numSig=len(sigList);
sigInd=np.arange(numSig);

plotGridMain=gs.GridSpec(numSig, 1);

In [35]:
pages=pdf.PdfPages("/data/gergely/workingData/raphe/run_stop_rewarded_residual.pdf");
for expType in expTypeList:
    for cd in cdList:
        fig=pp.figure(figsize=(numSig*2, numSig*numSig*2));
        fig.suptitle(str((expType, cd)));
        for resSig, i in zip(sigList, sigInd):
            plotGrid=gs.GridSpecFromSubplotSpec(numSig, numSig, 
                                                subplot_spec=plotGridMain[i, 0]);
            for rwdSig, j in zip(sigList, sigInd):
                for unRDSig, k in zip(sigList, sigInd):
                    ax=fig.add_subplot(plotGrid[j, k]);
                    numCells=0;
                    for traceKey in traceKeys:
                        try:
                            trace=plotTraces[traceKey].loc[(expType, cd,
                                                          resSig, unRDSig, rwdSig)].iloc[0];
                            numCells=trace["numSamples"];
                        except KeyError:
                            continue;
                        
                        ax.plot(trace["time"], trace["act"], lw=1, alpha=0.6, 
                                color=plotColors[traceKey]);
                        ax.fill_between(trace["time"], trace["act"]+trace["actSEM"],
                                        trace["act"]-trace["actSEM"], lw=0,
                                        alpha=0.4, color=plotColors[traceKey]);
                    
                    ax.axvline(0, lw=0.5, color="k", alpha=0.2, ls=(0, (5, 5)));
                    ax.set_ylim([-0.5, 1.5]);
                    ax.set_yticks([-0.5, 0, 0.5, 1, 1.5]);
                    ax.set_xticks([]);
                    ax.set_title(str(("rwd:"+str(rwdSig), 
                                      "unrwd:"+str(unRDSig), 
                                      "n="+str(numCells))));
                    plotTools.formatAxes(ax, "", "");
        
        pages.savefig(fig);
        pp.close(fig);
pages.close();

In [36]:
sigColors=["lightgrey", "r", "c"];
sigNums=plotROIs.groupby(["expType", "residualDiffSig", "diffSigRD", "diffSigURD"]).size();

In [37]:
pages=pdf.PdfPages("/data/gergely/workingData/raphe/run_stop_rewarded_residual_fractions.pdf");
for expType in expTypeList:
    fig=pp.figure(figsize=(3, numSig*3));
    fig.suptitle(expType);
    for resSig, i in zip(sigList, sigInd):
        ax=fig.add_subplot(plotGridMain[i, 0]);
        
        for rwdSig, j in zip(sigList, sigInd):
            cellN=0;
            for unRDSig, k in zip(sigList, sigInd):
                try:
                    print((expType, resSig, rwdSig, unRDSig))
                    sigN=sigNums.loc[(expType, resSig, rwdSig, unRDSig)];
                except:
                    continue;
                
                ax.bar(j, sigN, width=0.6, bottom=cellN, 
                       color=sigColors[unRDSig], lw=0, align="center");
                cellN=cellN+sigN;
        
        ax.set_title("residual sig: "+str(resSig));
        ax.set_xlim([-0.5, numSig-0.5]);
        ax.set_xticks(sigInd);
        ax.set_xticklabels(map(lambda n: str(n), sigList));
        plotTools.formatAxes(ax, "rewarded RS sig", "num cells-days");
        
    pages.savefig(fig);
    pp.close(fig);
pages.close();

('randomForaging', -1, -1, -1)
('randomForaging', -1, -1, 0)
('randomForaging', -1, -1, 1)
('randomForaging', -1, 0, -1)
('randomForaging', -1, 0, 0)
('randomForaging', -1, 0, 1)
('randomForaging', -1, 1, -1)
('randomForaging', -1, 1, 0)
('randomForaging', -1, 1, 1)
('randomForaging', 0, -1, -1)
('randomForaging', 0, -1, 0)
('randomForaging', 0, -1, 1)
('randomForaging', 0, 0, -1)
('randomForaging', 0, 0, 0)
('randomForaging', 0, 0, 1)
('randomForaging', 0, 1, -1)
('randomForaging', 0, 1, 0)
('randomForaging', 0, 1, 1)
('randomForaging', 1, -1, -1)
('randomForaging', 1, -1, 0)
('randomForaging', 1, -1, 1)
('randomForaging', 1, 0, -1)
('randomForaging', 1, 0, 0)
('randomForaging', 1, 0, 1)
('randomForaging', 1, 1, -1)
('randomForaging', 1, 1, 0)
('randomForaging', 1, 1, 1)
